In [6]:
import random
import openai
import time
import json
import argparse
import tiktoken
from vllm import LLM
from vllm.lora.request import LoRARequest
from openai import OpenAI
# from .autonotebook import tqdm as notebook_tqdm
# from langchain_community.llms import VLLM

openai_api_key = "EMPTY"
openai_api_base = "http://babel-3-13:8770/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
# sampling_params = SamplingParams(temperature=0.8, top_p=0.95)
chat_response = client.chat.completions.create(
    model=client.models.list().data[0].id,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Tell me a joke."},
    ],
    temperature=0.0
)
print("Chat response:", chat_response)

Chat response: ChatCompletion(id='cmpl-d317874eeda84771bcab2cb4d459281b', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="  Sure thing! Here's one for you:\n\nWhy couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!\n\nHope that brought a smile to your face! Is there anything else I can help you with?", role='assistant', function_call=None, tool_calls=None))], created=2688012, model='/data/datasets/models/huggingface/meta-llama/Llama-2-13b-chat-hf/', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=60, prompt_tokens=35, total_tokens=95))


In [ ]:
output = [RequestOutput(request_id=0, prompt='I want you act as an answer judge. Given a question and an answer, your objective is to determine if the provided answer contains non-factual or hallucinated information. You SHOULD give your judgement based on the following hallucination types and the world knowledge.\n\nYou are trying to determine if the answer misunderstands the question context and intention.\n#Question#: What is a rare breed of dog that was derived as a variant of Rat Terrier, Shiloh Shepherd dog or American Hairless Terrier?\n#Answer#: American Hairless Terrier\n#Your Judgement#: No\n\nYou are trying to determine if there is a factual contradiction between the answer and the world knowledge. Some information in the answer might be fabricated.\n#Question#: Are the New Orleans Outfall Canals the same length as the Augusta Canal?\n#Answer#: No, the New Orleans Outfall Canals and the Augusta Canal are not the same length. The Orleans Canal is approximately 3.6 miles (5.8 kilometers) long while the Augusta Canal is approximately 7 miles (11.3 kilometers) long.\n#Your Judgement#: Yes\n#Question#: What U.S Highway gives access to Zilpo Road, and is also known as Midland Trail?\n#Answer#: U.S Highway 70\n#Your Judgement#: Yes\n\nYou are trying to determine if the answer is too general or too specific to answer the question at an appropriate level of specificity.\n#Question#: What genre do Superheaven and Oceansize belong to?\n#Answer#: Superheaven and Oceansize belong to the rock genre.\n#Your Judgement#: No\n#Question#: What profession do Kōbō Abe and Agatha Christie share?\n#Answer#: Playwright.\n#Your Judgement#: No\n\nYou are trying to determine if the answer can be correctly inferred from the knowledge.\n#Question#: Which band has more members, Muse or The Raconteurs?\n#Answer#: Muse has more members than The Raconteurs.\n#Your Judgement#: Yes\n#Question#: Which is currently more valuable, Temagami-Lorrain Mine or Meadowbank Gold Mine?\n#Answer#: Meadowbank Gold Mine, since Meadowbank Gold Mine is still producing gold and the TemagamiLorrain Mine has been inactive for years.\n#Your Judgement#: No\n\nYou should try your best to determine if the answer contains non-factual or hallucinated information according to the above hallucination types. The answer you give MUST be \\"Yes\\" or \\"No\\"".\n\n#Question#: Which magazine was started first Arthur\'s Magazine or First for Women?\n#Answer#: First for Women was started first.\n#Your Judgement#:', prompt_token_ids=[1, 306, 864, 366, 1044, 408, 385, 1234, 16833, 29889, 11221, 263, 1139, 322, 385, 1234, 29892, 596, 12091, 338, 304, 8161, 565, 278, 4944, 1234, 3743, 1661, 29899, 17028, 950, 470, 12713, 1682, 262, 630, 2472, 29889, 887, 317, 8187, 29965, 10249, 2367, 596, 6577, 29887, 882, 2729, 373, 278, 1494, 12713, 1682, 3381, 4072, 322, 278, 3186, 7134, 29889, 13, 13, 3492, 526, 1811, 304, 8161, 565, 278, 1234, 19818, 1689, 29879, 278, 1139, 3030, 322, 16392, 29889, 13, 29937, 16492, 29937, 29901, 1724, 338, 263, 10812, 2078, 287, 310, 11203, 393, 471, 10723, 408, 263, 17305, 310, 17450, 5061, 4336, 29892, 1383, 309, 1148, 2296, 561, 2018, 11203, 470, 3082, 379, 1466, 2222, 5061, 4336, 29973, 13, 29937, 22550, 29937, 29901, 3082, 379, 1466, 2222, 5061, 4336, 13, 29937, 10858, 8660, 29887, 882, 29937, 29901, 1939, 13, 13, 3492, 526, 1811, 304, 8161, 565, 727, 338, 263, 2114, 950, 23949, 1546, 278, 1234, 322, 278, 3186, 7134, 29889, 3834, 2472, 297, 278, 1234, 1795, 367, 18187, 630, 29889, 13, 29937, 16492, 29937, 29901, 4683, 278, 1570, 26884, 4451, 11950, 1815, 1338, 278, 1021, 3309, 408, 278, 3111, 29874, 22642, 29973, 13, 29937, 22550, 29937, 29901, 1939, 29892, 278, 1570, 26884, 4451, 11950, 1815, 1338, 322, 278, 3111, 29874, 22642, 526, 451, 278, 1021, 3309, 29889, 450, 26884, 22642, 338, 14235, 29871, 29941, 29889, 29953, 7800, 313, 29945, 29889, 29947, 20052, 2699, 29897, 1472, 1550, 278, 3111, 29874, 22642, 338, 14235, 29871, 29955, 7800, 313, 29896, 29896, 29889, 29941, 20052, 2699, 29897, 1472, 29889, 13, 29937, 10858, 8660, 29887, 882, 29937, 29901, 3869, 13, 29937, 16492, 29937, 29901, 1724, 501, 29889, 29903, 22080, 4076, 2130, 304, 796, 309, 1129, 9321, 29892, 322, 338, 884, 2998, 408, 13370, 1049, 3201, 309, 29973, 13, 29937, 22550, 29937, 29901, 501, 29889, 29903, 22080, 29871, 29955, 29900, 13, 29937, 10858, 8660, 29887, 882, 29937, 29901, 3869, 13, 13, 3492, 526, 1811, 304, 8161, 565, 278, 1234, 338, 2086, 2498, 470, 2086, 2702, 304, 1234, 278, 1139, 472, 385, 8210, 3233, 310, 2702, 537, 29889, 13, 29937, 16492, 29937, 29901, 1724, 16151, 437, 5670, 354, 3496, 322, 438, 346, 550, 675, 6852, 304, 29973, 13, 29937, 22550, 29937, 29901, 5670, 354, 3496, 322, 438, 346, 550, 675, 6852, 304, 278, 7679, 16151, 29889, 13, 29937, 10858, 8660, 29887, 882, 29937, 29901, 1939, 13, 29937, 16492, 29937, 29901, 1724, 6351, 437, 476, 30099, 29890, 30099, 319, 915, 322, 4059, 493, 29874, 2819, 347, 6232, 29973, 13, 29937, 22550, 29937, 29901, 7412, 29893, 1266, 29889, 13, 29937, 10858, 8660, 29887, 882, 29937, 29901, 1939, 13, 13, 3492, 526, 1811, 304, 8161, 565, 278, 1234, 508, 367, 5149, 10115, 1127, 515, 278, 7134, 29889, 13, 29937, 16492, 29937, 29901, 8449, 3719, 756, 901, 5144, 29892, 13301, 470, 450, 6981, 535, 371, 1295, 29973, 13, 29937, 22550, 29937, 29901, 13301, 756, 901, 5144, 1135, 450, 6981, 535, 371, 1295, 29889, 13, 29937, 10858, 8660, 29887, 882, 29937, 29901, 3869, 13, 29937, 16492, 29937, 29901, 8449, 338, 5279, 901, 21114, 29892, 6789, 351, 4479, 29899, 29931, 272, 6038, 26833, 470, 341, 1479, 340, 9157, 6650, 26833, 29973, 13, 29937, 22550, 29937, 29901, 341, 1479, 340, 9157, 6650, 26833, 29892, 1951, 341, 1479, 340, 9157, 6650, 26833, 338, 1603, 20811, 7684, 322, 278, 6789, 351, 4479, 29931, 272, 6038, 26833, 756, 1063, 297, 4925, 363, 2440, 29889, 13, 29937, 10858, 8660, 29887, 882, 29937, 29901, 1939, 13, 13, 3492, 881, 1018, 596, 1900, 304, 8161, 565, 278, 1234, 3743, 1661, 29899, 17028, 950, 470, 12713, 1682, 262, 630, 2472, 5034, 304, 278, 2038, 12713, 1682, 3381, 4072, 29889, 450, 1234, 366, 2367, 341, 17321, 367, 13218, 8241, 5931, 470, 13218, 3782, 5931, 1642, 13, 13, 29937, 16492, 29937, 29901, 8449, 14853, 471, 4687, 937, 11498, 29915, 29879, 17880, 470, 3824, 363, 10152, 29973, 13, 29937, 22550, 29937, 29901, 3824, 363, 10152, 471, 4687, 937, 29889, 13, 29937, 10858, 8660, 29887, 882, 29937, 29901], prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' Yes\n\nPlease let me know if the answer contains non-factual or', token_ids=[3869, 13, 13, 12148, 1235, 592, 1073, 565, 278, 1234, 3743, 1661, 29899, 17028, 950, 470], cumulative_logprob=-2.720620002911801, logprobs=None, finish_reason=length)], finished=True, lora_request=LoRARequest(lora_name='dpo_adapter', lora_int_id=1, lora_local_path='/data/user_data/wenkail/llama_finetune_13b_persona/dpo_checkpoint/istj/checkpoint-12000'))]

In [7]:
llm = LLM(model=client.models.list().data[0].id, enable_lora=True)
prompts = [
     "[user] Write a SQL query to answer the question based on the table schema.\n\n context: CREATE TABLE table_name_74 (icao VARCHAR, airport VARCHAR)\n\n question: Name the ICAO for lilongwe international airport [/user] [assistant]",
     "[user] Write a SQL query to answer the question based on the table schema.\n\n context: CREATE TABLE table_name_11 (nationality VARCHAR, elector VARCHAR)\n\n question: When Anchero Pantaleone was the elector what is under nationality? [/user] [assistant]",
]
outputs = llm.generate(
    prompts,
    temperature=0,
    max_tokens=256,
    stop=["[/assistant]"],
    lora_request=LoRARequest("sql_adapter", 1, sql_lora_path)
)


OSError: Incorrect path_or_model_id: '/data/datasets/models/huggingface/meta-llama/Llama-2-13b-chat-hf/'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [19]:
chat_response.choices[0].message.content.strip()

"Sure thing! Here's one for you:\n\nWhy couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!\n\nHope that brought a smile to your face! Is there anything else I can help you with?"

In [ ]:
client.models.list().data[0].id

In [26]:
client.models.list().data[0]

Model(id='/data/datasets/models/huggingface/meta-llama/Llama-2-13b-chat-hf/', created=1707879231, object='model', owned_by='vllm', root='/data/datasets/models/huggingface/meta-llama/Llama-2-13b-chat-hf/', parent=None, permission=[{'id': 'modelperm-8aff986eace5414c9c8e84e63b94ce93', 'object': 'model_permission', 'created': 1707879231, 'allow_create_engine': False, 'allow_sampling': True, 'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 'allow_fine_tuning': False, 'organization': '*', 'group': None, 'is_blocking': False}])